<a href="https://colab.research.google.com/github/gvpmoneesha/QR-Code/blob/main/research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! ls

drive  sample_data


### Import Libraries

In [ ]:
import pandas as pd

#Tensorflow libraries
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# System libraries
from pathlib import Path
import os.path


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

--2025-07-30 07:04:27--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-07-30 07:04:27 (25.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



### Load and Transform Data

In [ ]:
TARGET_SIZE = (224, 224)

In [ ]:
# Walk through each directory
dataset = "drive/MyDrive/research/Train"
walk_through_dir(dataset)

There are 7 directories and 0 images in 'drive/MyDrive/research/Train'.
There are 0 directories and 250 images in 'drive/MyDrive/research/Train/Bacterial gill disease'.
There are 0 directories and 250 images in 'drive/MyDrive/research/Train/Bacterial diseases - Aeromoniasis'.
There are 0 directories and 250 images in 'drive/MyDrive/research/Train/Bacterial Red disease'.
There are 0 directories and 250 images in 'drive/MyDrive/research/Train/Parasitic diseases'.
There are 0 directories and 250 images in 'drive/MyDrive/research/Train/Fungal diseases Saprolegniasis'.
There are 0 directories and 250 images in 'drive/MyDrive/research/Train/Viral diseases White tail disease'.
There are 0 directories and 250 images in 'drive/MyDrive/research/Train/Healthy Fish'.


### Create a Dataframe for Image Data

In [ ]:
def convert_path_to_df(dataset):
    image_dir = Path(dataset)

    # Get filepaths and labels
    filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.jpeg')) + list(image_dir.glob(r'**/*.PNG'))

    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    image_df = pd.concat([filepaths, labels], axis=1)
    return image_df

image_df = convert_path_to_df(dataset)


### Data split into Train, Validation, and Test

In [ ]:
from sklearn.model_selection import train_test_split

# Train-test split (80% train, 20% test)
train_df, test_df = train_test_split(image_df, test_size=0.2, stratify=image_df['Label'], random_state=42)

# Further split training into train and validation (80% of 80% = 64% train, 16% val)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Label'], random_state=42)

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 1099, Validation: 275, Test: 344


### Pre Processing

In [ ]:
# Train: use data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Validation and Test: only rescaling
val_test_datagen = ImageDataGenerator(rescale=1./255)

### Create Data Generator

In [ ]:
train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=True,
    batch_size=32
)

val_gen = val_test_datagen.flow_from_dataframe(
    val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False,
    batch_size=32
)

test_gen = val_test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False,
    batch_size=32
)


Found 1099 validated image filenames belonging to 7 classes.
Found 275 validated image filenames belonging to 7 classes.
Found 344 validated image filenames belonging to 7 classes.


In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, ResNet152
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Get number of classes
num_classes = len(train_gen.class_indices)

# Function to build model
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model



In [ ]:
base_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg = build_model(base_vgg)

model_vgg.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history_vgg = model_vgg.fit(train_gen, validation_data=val_gen, epochs=10)

print("VGG16 Accuracy:", history_vgg.history['val_accuracy'][-1])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2567s 73s/step - accuracy: 0.1496 - loss: 2.8476 - val_accuracy: 0.1382 - val_loss: 1.9458
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2507s 72s/step - accuracy: 0.1506 - loss: 1.9474 - val_accuracy: 0.1418 - val_loss: 1.9459
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2541s 72s/step - accuracy: 0.1203 - loss: 1.9463 - val_accuracy: 0.1418 - val_loss: 1.9459
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2531s 71s/step - accuracy: 0.1379 - loss: 1.9460 - val_accuracy: 0.1418 - val_loss: 1.9459
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2497s 71s/step - accuracy: 0.1603 - loss: 1.9460 - val_accuracy: 0.1418 - val_loss: 1.9459
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2531s 72s/step - accuracy: 0.1448 - loss: 1.9460 - val_accuracy: 0.1418 - val_loss: 1.9459
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2492s 71s/step - accuracy: 0.1504 - loss: 1.9459 - val_accuracy: 0.1418 - val_loss: 1.9459
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2515s 72s/step - accuracy: 0.1378 - loss: 1.9461 - val_accu

In [ ]:
base_res50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_res50 = build_model(base_res50)

model_res50.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history_res50 = model_res50.fit(train_gen, validation_data=val_gen, epochs=10)

print("ResNet50 Accuracy:", history_res50.history['val_accuracy'][-1])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1062s 29s/step - accuracy: 0.3695 - loss: 1.8862 - val_accuracy: 0.1418 - val_loss: 69.1875
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 968s 28s/step - accuracy: 0.3976 - loss: 1.6377 - val_accuracy: 0.1564 - val_loss: 2.3489
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 971s 28s/step - accuracy: 0.4980 - loss: 1.4140 - val_accuracy: 0.1418 - val_loss: 5.8170
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 960s 27s/step - accuracy: 0.5524 - loss: 1.2821 - val_accuracy: 0.1709 - val_loss: 2.3970
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 963s 28s/step - accuracy: 0.6192 - loss: 1.1146 - val_accuracy: 0.1455 - val_loss: 81.6132
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 962s 28s/step - accuracy: 0.6483 - loss: 0.9810 - val_accuracy: 0.1418 - val_loss: 8.6500
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 985s 28s/step - accuracy: 0.7019 - loss: 0.9112 - val_accuracy: 0.1418 - val_loss: 2.6314
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 965s 28s/ste

In [ ]:
base_res152 = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_res152 = build_model(base_res152)

model_res152.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history_res152 = model_res152.fit(train_gen, validation_data=val_gen, epochs=10)

print("ResNet152 Accuracy:", history_res152.history['val_accuracy'][-1])
